In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
df_tran = pd.read_csv('../dataset/tran.txt',names=['primary-date','time','approximate-indicator','secondary-date','approximate-indicator',
  'transaction-ID','player','type','from-team','from-league','to-team','to-league','draft-type','draft-round','pick-number','info'])
df_playerid = pd.read_csv('../dataset/playerid_list.csv',names=['LASTNAME','FIRSTNAME','PLAYERID','DAVENPORTCODE','MLBCODE','player'],skiprows=1);
df_master = pd.read_csv('../dataset/Master.csv')
df_batstat = pd.read_csv('../dataset/war_daily_bat.txt')
df_pitchstat = pd.read_csv('../dataset/war_daily_pitch.txt')
pd.set_option('display.max_columns', 60)

# Data Cleanup
# Recontructing invalid dates
df_tran['primary-date'] = df_tran['primary-date'].astype('str')
df_tran['primary-date'] = df_tran['primary-date'].str.replace(r'\A(\d{4})[0]{4}', r'\g<1>0101')
df_tran['primary-date'] = df_tran['primary-date'].str.replace(r'\A(\d{4})(..)[0]{2}', r'\g<1>\g<2>01')

#Extracting only trades from transaction dataframe
df_tran = df_tran[df_tran['type'] == "T "]
df_tran = df_tran[df_tran['to-league'].isin(['AL','NL'])]
df_tran = df_tran[df_tran['from-league'].isin(['AL','NL'])]

df_tran['primary-date'] = pd.to_datetime(df_tran['primary-date'], format='%Y%m%d')
df_tran['tranYear'] = df_tran['primary-date'].apply(lambda x : int(x.year))

C:\Users\arpith\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_master['player_ID'] = df_master['bbrefID']
df_batstat = pd.merge(df_batstat,df_master[['player_ID','retroID']],on='player_ID',how='inner')
df_pitchstat = pd.merge(df_batstat,df_master[['player_ID','retroID']],on='player_ID',how='inner')

In [3]:
df_batstat.head()

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,retroID
0,David Aardsma,22.0,430911.0,aardsda01,2004,SFG,1,NL,0,11,10.7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.1,0.1,0.1,0.1,0.01,0.01,0.00,0.01,0.00,0.01,0.0,300000.0,Y,4.67546,4.67092,0.08651,4.67092,1.891,1.890,0.5005,0.5005,0.5005,0.5000,NaN,0.000,0.000,aardd001
1,David Aardsma,24.0,430911.0,aardsda01,2006,CHC,1,NL,3,45,53.0,-0.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.21,0.46,0.0,-0.2,-0.2,-0.2,0.2,-0.02,-0.02,0.00,-0.02,0.00,-0.02,0.0,NaN,Y,4.86164,4.86675,0.09085,4.86467,1.912,1.913,0.4995,0.4995,0.5005,0.4998,-100.0,0.694,0.896,aardd001
2,David Aardsma,25.0,430911.0,aardsda01,2007,CHW,1,AL,0,25,32.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11,0.00,0.0,0.1,0.1,0.1,0.1,0.01,0.01,0.02,0.01,0.02,0.01,0.0,387500.0,Y,4.86335,4.85895,0.08422,4.85895,1.912,1.912,0.5004,0.5004,0.5004,0.5000,NaN,0.000,0.000,aardd001
3,David Aardsma,26.0,430911.0,aardsda01,2008,BOS,1,AL,1,47,48.7,-0.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.22,0.14,0.0,0.1,0.1,0.1,0.2,0.00,0.00,0.05,0.00,0.05,0.00,0.0,403250.0,Y,4.70549,4.70400,0.08092,4.70321,1.894,1.894,0.5001,0.5001,0.5005,0.4999,-100.0,0.345,0.434,aardd001
4,David Aardsma,27.0,430911.0,aardsda01,2009,SEA,1,AL,0,73,71.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.32,0.00,0.0,0.3,0.3,0.3,0.3,0.03,0.03,0.06,0.03,0.06,0.03,0.0,419000.0,Y,4.80227,4.79788,0.08302,4.79788,1.905,1.905,0.5004,0.5004,0.5004,0.5000,NaN,0.000,0.000,aardd001


In [ ]:
#Calculates the absolute difference between two sides of a trade by summing the WAR values of each player on each side for 
#the year in which the trade took place
def trade_value(tranID,metric):
    td = df_tran[(df_tran['transaction-ID'] == tranID)]
    teams = td['from-team'].unique()
    val = 0
    player_vals = []
    for x in teams:
        for index, row in td.iterrows():
            val = 0
            if (row['from-team'] == x):
                if pd.isnull(row['player']):
                    # filter out cash transactions or yet to be named player transaction 
                    continue
                player_df = df_batstat[(df_batstat['retroID'] == row['player'])]
                player_df_year = player_df[(df_batstat['year_ID'] == row['tranYear'])]
                if not player_df_year.empty:
                    val += float(player_df_year[metric].iloc[0])
                elif not player_df.empty:
                    # take a random value if we don't have war for that transaction year, put logic to change this to nearest year
                    val += float(player_df['WAR'].iloc[0])
                else:
                    ## figure out a way to find war for these players
                    val += 0
        player_vals.append(val)
    return abs(player_vals[1]-player_vals[0])

In [ ]:
#Calculates the absolute difference between two sides of a trade by summing the WAR values of each player on each side one
# year after the trade took place
def trade_value_next(tranID,metric):
    td = df_tran[(df_tran['transaction-ID'] == tranID)]
    teams = td['from-team'].unique()
    val = 0
    player_vals = []
    for x in teams:
        for index, row in td.iterrows():
            val = 0
            if (row['from-team'] == x):
                if pd.isnull(row['player']):
                    # filter out cash transactions or yet to be named player transaction 
                    continue
                player_df = df_batstat[(df_batstat['retroID'] == row['player'])]
                player_df_year = player_df[(df_batstat['year_ID'] == row['tranYear'])]
                player_df_year_next = player_df[(df_batstat['year_ID'] == (row['tranYear'] + 1))]
                if not player_df_year_next.empty:
                    # get the value of metric one year after trade
                    val += float(player_df_year_next[metric].iloc[0])
                elif not player_df_year.empty:
                    # if can't find above value assume its the same as the last year's value 
                    val += float(player_df_year[metric].iloc[0])
                elif not player_df.empty:
                    # take a random value if we don't have war for that transaction year, put logic to change this to nearest year
                    val += float(player_df[metric].iloc[0])
                else:
                    ## figure out a way to find war for these players
                    val += 0
        player_vals.append(val)
    return (abs(player_vals[1]-player_vals[0]),tranID)

In [ ]:
#Calculating the trade value difference for each transaction using metric WAR,WAA,PA,Salary & TA
trade_val_list_WAR = []
transactions = df_tran['transaction-ID'].unique()
for x in transactions:
    trade_val_list_WAR.append(trade_value(x,'WAR'))

In [ ]:
# Scatter plot for Wins Above Replacement
x_pos = np.arange(0,len(trade_val_list_WAR))
plt.figure(figsize=(15, 8))
plt.title("Scatter Plot for difference in WAR values of players at the time of trade")
plt.xlabel("WAR")
plt.scatter(x_pos, trade_val_list_WAR)
plt.show

In [ ]:
trade_val_list_WAR_next = []
transactions = df_tran['transaction-ID'].unique()
for x in transactions:
    trade_val_list_WAR_next.append(trade_value_next(x,'WAR'))
    
#trade_val_list
new_trade_val_list_WAR_next =[]
for i in trade_val_list_WAR_next:
    new_trade_val_list_WAR_next.append(i[0])
    
x_pos = np.arange(0,len(new_trade_val_list_WAR_next))
plt.figure(figsize=(15, 8))
plt.title("Scatter Plot for difference in WAR values of players one year after the trade")
plt.xlabel("WAR")
plt.scatter(x_pos, new_trade_val_list_WAR_next)
plt.show

In [ ]:
# sorting values based on difference in WAR scores
sorted_trade_df = sorted(trade_val_list_WAR_next,key=lambda x:x[0],reverse=True)
sorted_trade_df

In [ ]:
df_batstat.head()

In [ ]:
df_batstat_sort = df_batstat.sort('age')
df_batstat_sort = df_batstat_sort[pd.notnull(df_batstat_sort['age'])]

In [ ]:
df_batstat_sort.head()